# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print("The current working directory:", os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

The current working directory: /home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# creating a smaller event data csv file called event_datafile_new csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
file = 'event_datafile_new.csv'
with open(file, 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra. 

The CSV file titled <font color=red>event_datafile_new.csv</font> is located within the Workspace directory. The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e: 
    print(e)    

### Create Keyspace

In [6]:
create_keyspace_q = """
    CREATE KEYSPACE sparkify_event WITH replication = {
        'class': 'SimpleStrategy', 
        'replication_factor' : 1
        };
    """

# Create a Keyspace - sparkify_event
try:
    session.execute("DROP KEYSPACE IF EXISTS sparkify_event")
    session.execute(create_keyspace_q)
except Exception as e:
    print(e)

### Set Keyspace

In [7]:
# Set KEYSPACE to sparkify_event
try:
    session.set_keyspace('sparkify_event')
except Exception as e:
    print(e)

## Three types of queries are conbsided:
1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### Creating artist_playlist_session
- In Query 1, the search is based on the session_id and item_in_session. The partition keys should be a tuple of (session_id, item_in_session). 
- With the session_id and item_in_session, we should be able to identify the record in the playlist. So, they also form the primary key.
- The table should contains three columns of interest, artist_name, song_title, and length.
- Since we are interested in artist_name, song_title, and the length, the table can be named as artist_playlist_session. 

#### Design the Query for Query 1.

In [8]:
query_1 = """
    SELECT artist_name, song_title, length from artist_playlist_session where 
        session_id=338 and item_in_session=4;
"""                   

In [9]:
# Test if there is duplicated rows.
df = pd.read_csv("./event_datafile_new.csv")
is_duplicated = np.any(df[['sessionId', 'itemInSession']].duplicated())
if is_duplicated:
    print("There is duplicated rows!")
else:
    print("There is no duplicated rows.")

There is no duplicated rows.


#### Create artist_playlist_session Table and Insert data into Table

In [10]:
# Create artist_playlist_session table
try: 
    session.execute("DROP TABLE IF EXISTS artist_playlist_session")
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS artist_playlist_session "
query += """
    (session_id int,
     item_in_session int,
     artist_name text,
     song_title text,
     length float,
     PRIMARY KEY (session_id, item_in_session)
        )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)
    
# Here we read the file line by line, and insert the related data into the database.
query = "INSERT INTO artist_playlist_session "
query += """
    (session_id, item_in_session, artist_name, song_title, length)
     """
query += " VALUES (%s, %s, %s, %s, %s);"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Run the SELECT to verify that the data have been inserted into each table

In [11]:
## Try query 1 to verify the data was entered into the table

try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Creating song_playlist_session
- In Query 2, the search is based on the user_id and session_id. The partition keys should be a tuple of them. 
- With the user_id and session_id, we can not uniquely identify the record in the playlist. We need  item_in_session in primary key to do so and make the song is sorted by item_in_session.
- The table should contains three columns of interest, artist_name, song_title, and first_name.
- Since we are interested in artist_name, song_title, and the user first name, the table can be named as song_playlist_session. 

#### Writing Query 2

In [12]:
query_2 = """
    SELECT artist_name, song_title, first_name FROM song_playlist_session 
        WHERE user_id=10 and session_id=182;
"""

#### Creating song_playlist_session and Inserting data into it

In [13]:
# Create song_playlist_session
try: 
    session.execute("DROP TABLE IF EXISTS song_playlist_session")
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query += """
    (user_id int,
     session_id int,
     item_in_session int,    
     artist_name text, 
     song_title text,
     first_name text,
     PRIMARY KEY ((user_id, session_id), item_in_session)
        )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "INSERT INTO song_playlist_session "
query += """
    (user_id, session_id, item_in_session, artist_name, song_title, first_name)
     """
query += " VALUES (%s, %s, %s, %s, %s, %s);"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1]))
        except Exception as e:
            print(e)

#### Run the SELECT to verify that the data have been inserted into each table

In [14]:
try:
    rows = session.execute(query_2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_title, row.first_name)

Down To The Bone Keep On Keepin' On Sylvie
Three Drives Greece 2000 Sylvie
Sebastien Tellier Kilometer Sylvie
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie


### Creating user_palylist_session Table
- In Query 3, the search is based on the song title, and it can be the partition key. 
- With the song title only, we can not uniquely identify the record in the playlist. We need user_id with song title to form a primary key to do so.
- The table should contains two columns of interest, first_name, last_name
- Since we are interested in user, the table can be named as song_playlist_session. 

#### Designing Query 3

In [15]:
query_3 = """
    SELECT first_name, last_name FROM user_playlist_session
        WHERE song_title='All Hands Against His Own';
"""

#### Creating Table user_playlist_session and Insert data into the Table

In [16]:
# Create music_history table
try: 
    session.execute("DROP TABLE IF EXISTS user_playlist_session")
except Exception as e:
    print(e)

query = "CREATE TABLE IF NOT EXISTS user_playlist_session "
query += """
    (song_title text, 
     user_id int,
     first_name text,
     last_name text,
     PRIMARY KEY (song_title, user_id)
        )
"""
try:
    session.execute(query)
except Exception as e:
    print(e)

query = "INSERT INTO user_playlist_session "
query += """
    (song_title, user_id, first_name, last_name)
     """
query += " VALUES (%s, %s, %s, %s);"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

#### Run Query 3 to Validate the Data Insertion

In [17]:
try:
    rows = session.execute(query_3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.first_name, row.last_name)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
try: 
    session.execute("DROP TABLE IF EXISTS music_history")
except Exception as e:
    print(e)

try:
    session.execute("DROP KEYSPACE IF EXISTS project2")
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()